# Bulldozers example

### Initialize & install modules

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install numpy
!pip install matplotlib
!pip install sklearn
!pip install torch
!pip install nvidia-ml-py3 beautifulsoup4 fastprogress
!pip install bottleneck dataclasses

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from fastai.tabular import *

# Load data

In [5]:
# Download 'Blue book Bulldozer' dataset from Kallgle
data_path = 'data/bulldozer/TrainAndValid.csv'

In [7]:
# When loading, make sure we parse 'saledate' column
df_raw = pd.read_csv(data_path, low_memory=False, parse_dates=['saledate'])

In [144]:
df = df_raw.copy()

### Define dependent variable

In [8]:
dep_var = 'SalePrice'

In [9]:
# Use log of salesprice instead of saleprice (as defined in the problem description)
df.SalePrice = np.log(df.SalePrice)

### Convert categorical entries

In [10]:
cat_names = ['UsageBand', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls']

In [145]:
sanitize_valid_chars = set('-_.abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

def sanitize(s):
    ''' Sanitize a string by only allowing "valid" characters '''
    return ''.join(c for c in str(s) if c in sanitize_valid_chars)

In [146]:
def rename_category_cols(df, prepend):
    ''' Rename dataFrame columns by prepending a string and sanitizing the name '''
    names = dict()
    for c in df.columns:
        name = f"{prepend}{sanitize(c)}"
        names[c] = name
    df.rename(columns=names, inplace=True)
    return df


In [170]:
def one_hot(df, max_categories=7):
    '''
    Change all columns in dataFrame having a cardinaliy less
    than 'max_categories' to 'one hot' encoding
    '''
    to_replace = dict()
    for c in df.columns:
        xi = df[c]
        if xi.dtype == 'O':
            values = xi.unique()
            count_cats = len(values)
            if count_cats <= max_categories:
                has_na = (xi.isna().sum() > 0)
                xi_dummies = pd.get_dummies(xi, dummy_na=has_na)
                to_replace[c] = rename_category_cols(xi_dummies, f"{c}_")
                # print(f"Column: {c}\n\tcount_cats: {count_cats}\n\tvalues: {values}\n\tNew columns: {xi_dummies.columns}")
    
    # Make all replacements: Drop old columns, join new ones
    df_new = df.copy()
    print(f"Before join: {df_new.columns}")
    for c in to_replace:
        df_new = df_new.join(to_replace[c])
    print(f"Before drop: {df_new.columns}\n\t{list(to_replace.keys())}")
    df_new.drop(list(to_replace.keys()), axis=1, inplace=True)
    return df_new


In [171]:
df_new = one_hot(df)

Before join: Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')
Before drop: Index(['SalesID', 'SalePrice

In [172]:
df_new.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'saledate',
       'fiModelDesc',
       ...
       'Differential_Type_Locking', 'Differential_Type_NoSpin',
       'Differential_Type_Standard', 'Differential_Type_nan',
       'Steering_Controls_CommandControl', 'Steering_Controls_Conventional',
       'Steering_Controls_FourWheelStandard', 'Steering_Controls_No',
       'Steering_Controls_Wheel', 'Steering_Controls_nan'],
      dtype='object', length=151)

In [173]:
df_new.dtypes

SalesID                                         int64
SalePrice                                     float64
MachineID                                       int64
ModelID                                         int64
datasource                                      int64
auctioneerID                                  float64
YearMade                                        int64
MachineHoursCurrentMeter                      float64
saledate                               datetime64[ns]
fiModelDesc                                    object
fiBaseModel                                    object
fiSecondaryDesc                                object
fiModelSeries                                  object
fiModelDescriptor                              object
fiProductClassDesc                             object
state                                          object
Transmission                                   object
Hydraulics                                     object
Tire_Size                   

NameError: name 'x' is not defined

### Split date into several features

In [ ]:
df = add_datepart(df, 'saledate')

In [ ]:
valid_idx = range(len(df)-12000, len(df))

### Add columns for 'NA'

# OLD CODE

In [ ]:
data = TabularDataBunch.from_df(data_path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names) 

In [ ]:
# Convert Data tabular bunsh back into data frame
# Reference https://forums.fast.ai/t/tabulardatabunch-to-pandas-dataframe/30947/6
def get_proc_df(tll):
    """Get processed xs and ys from a tabular `LabelList` with a single value for label such as FloatList.
       For example from `TabularDataBunch.train_ds`.
       :param tll: A tabular `LabelList`. 
       :returns: A tuple of `(x,y)` where `x` is a pandas `DataFrame` and `y` is a numpy array.
    """
    x_vals = np.concatenate([tll.x.codes, tll.x.conts], axis=1)
    x_cols = tll.x.cat_names + tll.x.cont_names
    x_df = pd.DataFrame(data=x_vals, columns=x_cols)[
            [c for c in tll.inner_df.columns if c in x_cols] ] # Retain order
    # Reconstruct ys to apply log if specified
    y_vals = np.array([i.obj for i in tll.y])
    return x_df, y_vals

In [ ]:
train_x_df, train_y_vals = get_proc_df(data.train_ds)
val_x_df, val_y_vals = get_proc_df(data.valid_ds)

# Sub-sample training set for speed
train_subset = 50_000
x_df = train_x_df[-train_subset:]
y = train_y_vals[-train_subset:]

In [ ]:
# Functions to show training results

def rmse(x, y): return math.sqrt(((x - y)**2).mean())

def print_score(m):
    ret = [rmse(m.predict(x_df), y), rmse(m.predict(val_x_df), val_y_vals), m.score(x_df, y), m.score(val_x_df, val_y_vals)]
    print(ret)
    return ret

In [ ]:
x_df.head()

In [ ]:
# Train model: Zero (exploratory analysis)
m = RandomForestRegressor(n_jobs=-1, n_estimators=1, max_depth=3, bootstrap=False)
m.fit(x_df, y)
print_score(m)


In [ ]:
# Export the tree to a graphviz 'dot' format
from sklearn.tree import export_graphviz
from IPython import display

str_tree = export_graphviz(m.estimators_[0],
                           out_file='tree.dot',
                           feature_names=x_df.columns,
                           filled=True,
                           rounded=True)

# Convert 'dot' to 'png'
!dot -Tpng 'tree.dot' -o 'tree.png'

# Show image
from IPython.display import Image
Image(filename='tree.png') 

In [ ]:
# Train model: 1
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
m.fit(x_df, y)
print_score(m)

In [ ]:
# Train model: 2
m = RandomForestRegressor(n_jobs=-1, n_estimators=100, min_samples_leaf=10, max_features=0.5)
m.fit(x_df, y)
print_score(m)

In [ ]:
preds = np.stack([t.predict(val_x_df) for t in m.estimators_])

In [ ]:
np.mean(preds[:,]), np.std(preds[:,0])

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=3, max_features=0.5)
%time m.fit(x_df, y)
print_score(m)

In [ ]:
def one_hot(x, max_categories=10):
    for c in x.columns:
        values = x_df[c].unique()
        count_cats = len(values)
        if count_cats <= max_categories: 
            print(f"{c}: {count_cats}\t{values}")

In [ ]:
values = x_df.ProductSize.unique()
len(values)

In [ ]:
one_hot(x_df)